In [50]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
warnings.filterwarnings('ignore')

In [119]:
df

,abstract,articleTitle,Journaltitle,volume,pubDate,authors,MeSh,citations,affiliations,keywords,IsReviewArticle
PMID,,,,,,,,,,,
35961771,Navigation through complex environments requir...,Coordination between eye movement and whisking...,eNeuro,1,2022-Aug-12,NaN,None,0,[],None,0
35961623,Recent evidence shows that genetic and environ...,Polygenic risk for schizophrenia as a moderato...,Progress in neuro-psychopharmacology & biologi...,1,2022-Aug-09,NaN,None,0,[],"Adversity, Bipolar disorder, Genetics, Psychos...",0
35961621,This study aimed to examine the prospective as...,The association of substance use with attainin...,Preventive medicine,1,2022-Aug-09,NaN,None,0,[],"Alcohol use, Cannabis use, Employment, Tobacco...",0
35961602,Psychosis presentation can be affected by gene...,Differences of affective and non-affective psy...,Journal of affective disorders,1,2022-Aug-09,NaN,None,0,[],"Bipolar disorder, Depression, First-episode, P...",0
35961582,Mental health and cognitive achievement are pa...,Brain structure and function show distinct rel...,Biological psychiatry. Cognitive neuroscience ...,1,2022-Aug-09,NaN,None,0,[],"cognition, genetics, mental health, polygenic ...",0
...,...,...,...,...,...,...,...,...,...,...,...
32848624,Hearing loss is the third most common chronic ...,Berbamine Analogs Exhibit Differential Protect...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,58,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","aminoglycoside, berbamine, hair cell, hearing ...",0
32848620,Tyrosine hydroxylase (Th) expression has previ...,Purkinje Cell-Specific Knockout of Tyrosine Hy...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,124,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","catecholamine, cerebellum, cognition, dopamine...",0
32848619,Myelin is a dynamic membrane that is important...,That's a Wrap! Molecular Drivers Governing Neu...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,48,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","Caspr, Nogo-A, PrPc, myelin, nogo receptor, pa...",1


In [142]:
data  = pd.read_csv('../raw_data/neuroscience_data_100000.csv').drop(columns=['Unnamed: 0']).set_index('PMID').dropna(subset=['abstract','articleTitle'])
df_clean_title = data[~data.articleTitle.str.contains("{")]
df = df_clean_title[~df_clean_title.abstract.str.contains("{")]
df = df[df.abstract != '.']
df


,abstract,articleTitle,Journaltitle,volume,pubDate,authors,MeSh,citations,affiliations,keywords,IsReviewArticle
PMID,,,,,,,,,,,
35961771,Navigation through complex environments requir...,Coordination between eye movement and whisking...,eNeuro,1,2022-Aug-12,NaN,None,0,[],None,0
35961623,Recent evidence shows that genetic and environ...,Polygenic risk for schizophrenia as a moderato...,Progress in neuro-psychopharmacology & biologi...,1,2022-Aug-09,NaN,None,0,[],"Adversity, Bipolar disorder, Genetics, Psychos...",0
35961621,This study aimed to examine the prospective as...,The association of substance use with attainin...,Preventive medicine,1,2022-Aug-09,NaN,None,0,[],"Alcohol use, Cannabis use, Employment, Tobacco...",0
35961602,Psychosis presentation can be affected by gene...,Differences of affective and non-affective psy...,Journal of affective disorders,1,2022-Aug-09,NaN,None,0,[],"Bipolar disorder, Depression, First-episode, P...",0
35961582,Mental health and cognitive achievement are pa...,Brain structure and function show distinct rel...,Biological psychiatry. Cognitive neuroscience ...,1,2022-Aug-09,NaN,None,0,[],"cognition, genetics, mental health, polygenic ...",0
...,...,...,...,...,...,...,...,...,...,...,...
32848625,The inferior colliculus (IC) is an auditory mi...,Developmentally Regulated Rebound Depolarizati...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,74,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","action potential, auditory system, brain slice...",0
32848624,Hearing loss is the third most common chronic ...,Berbamine Analogs Exhibit Differential Protect...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,58,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","aminoglycoside, berbamine, hair cell, hearing ...",0
32848620,Tyrosine hydroxylase (Th) expression has previ...,Purkinje Cell-Specific Knockout of Tyrosine Hy...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,124,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","catecholamine, cerebellum, cognition, dopamine...",0


In [143]:
df_abstract = df[['abstract']]
df_abstract

,abstract
PMID,
35961771,Navigation through complex environments requir...
35961623,Recent evidence shows that genetic and environ...
35961621,This study aimed to examine the prospective as...
35961602,Psychosis presentation can be affected by gene...
35961582,Mental health and cognitive achievement are pa...
...,...
32848625,The inferior colliculus (IC) is an auditory mi...
32848624,Hearing loss is the third most common chronic ...
32848620,Tyrosine hydroxylase (Th) expression has previ...


In [144]:
df_title = df[['articleTitle']]
df_title

,articleTitle
PMID,
35961771,Coordination between eye movement and whisking...
35961623,Polygenic risk for schizophrenia as a moderato...
35961621,The association of substance use with attainin...
35961602,Differences of affective and non-affective psy...
35961582,Brain structure and function show distinct rel...
...,...
32848625,Developmentally Regulated Rebound Depolarizati...
32848624,Berbamine Analogs Exhibit Differential Protect...
32848620,Purkinje Cell-Specific Knockout of Tyrosine Hy...


In [145]:
df_title.dtypes

articleTitle    object
dtype: object

In [146]:
def clean_txt(text):
    
    text = text.lower()
    
    for punctuation in string.punctuation:
        
        text = text.replace(punctuation,'')
              
    text = ''.join(char for char in text if not char.isdigit()) 
    
    tokenized_text = word_tokenize(text)
    stop_words = set(stopwords.words('english')) 
    tokenized_sentence_cleaned = [w for w in tokenized_text 
                                  if not w in stop_words]
    
    verb_lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "v")
              for word in tokenized_sentence_cleaned]
    
    noun_lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "n")  # n --> nouns
               for word in verb_lemmatized]
        
    return " ".join(noun_lemmatized)

In [147]:
df_title.articleTitle = df_title.articleTitle.astype(str).apply(clean_txt)

df_title

,articleTitle
PMID,
35961771,coordination eye movement whisk head fix mouse...
35961623,polygenic risk schizophrenia moderator associa...
35961621,association substance use attain employment am...
35961602,difference affective nonaffective psychosis ea...
35961582,brain structure function show distinct relatio...
...,...
32848625,developmentally regulate rebound depolarizatio...
32848624,berbamine analog exhibit differential protecti...
32848620,purkinje cellspecific knockout tyrosine hydrox...


In [148]:
df_abstract.abstract = df_abstract.abstract.astype(str).apply(clean_txt)
df_abstract

,abstract
PMID,
35961771,navigation complex environment require motor p...
35961623,recent evidence show genetic environmental ris...
35961621,study aim examine prospective association toba...
35961602,psychosis presentation affect genetic environm...
35961582,mental health cognitive achievement partly her...
...,...
32848625,inferior colliculus ic auditory midbrain struc...
32848624,hear loss third common chronic health conditio...
32848620,tyrosine hydroxylase th expression previously ...


In [149]:
tfidf_vectorizer_abstract = TfidfVectorizer()
tfidf_vectorizer_title =TfidfVectorizer()

In [150]:
tfidf_abstract = tfidf_vectorizer_abstract.fit_transform(df_abstract.abstract)
tfidf_abstract

<88070x197778 sparse matrix of type '<class 'numpy.float64'>'
	with 8007116 stored elements in Compressed Sparse Row format>

In [151]:
tfidf_title = tfidf_vectorizer_title.fit_transform(df_title.articleTitle)
tfidf_title

<88070x50248 sparse matrix of type '<class 'numpy.float64'>'
	with 870832 stored elements in Compressed Sparse Row format>

In [152]:
input_topic = 'brain injury on mice through imaging'

In [153]:
cleaned_input = pd.DataFrame({'Predict searc':clean_txt(input_topic)},index=[1])
cleaned_input

,Predict searc
1,brain injury mouse image


In [134]:
input_tfidf_title = tfidf_vectorizer_title.transform(cleaned_input)
input_tfidf_title

<1x50250 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [135]:
input_tfidf_abstract = tfidf_vectorizer_abstract.transform(cleaned_input)
input_tfidf_abstract

<1x197778 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [136]:
n_neighbors = 10

KNN_t = NearestNeighbors(n_neighbors)
KNN_t.fit(tfidf_title)

NNs_t = KNN_t.kneighbors(input_tfidf_title,return_distance=True)

df.iloc[list(NNs_t[1][0]), :]

,abstract,articleTitle,Journaltitle,volume,pubDate,authors,MeSh,citations,affiliations,keywords,IsReviewArticle
PMID,,,,,,,,,,,
35814496,Understanding how the brain learns may lead to...,Neurons learn by predicting future activity.,Nature machine intelligence,4,2022-Jan,"McNaughton Ryan, Huo Yuda, Li Guicai, Ioschpe ...",None,65,"MA, USA., MA, USA., MA, USA., MA, USA., MA, US...",None,0
34194469,Autism spectrum disorder (ASD) refers to a wid...,Predicting the Risk Genes of Autism Spectrum D...,Frontiers in genetics,12,2021,"Dib Rama, Gervais Nicole J, Mongrain Valérie",None,72,"QC, Canada., QC, Canada., QC, Canada.","autism spectrum disorders, feature selection, ...",0
34792465,Two neural circuits control the release of vas...,Predicting changes in osmolality.,eLife,10,2021-11-18,"Robinson Gail A, Tjokrowijoto Priscilla, Cesli...","Drinking, Osmolar Concentration, Water",10,"China. wongcb@unimelb.edu.au., China. wongcb@u...","feedforward, fiber photometry, medicine, mouse...",0
33772353,Amyotrophic lateral sclerosis (ALS) is a progr...,Cognitive dysfunction in amyotrophic lateral s...,Neurological sciences : official journal of th...,42,2021-Jun,"Pyasik Maria, Pia Lorenzo","Amyotrophic Lateral Sclerosis, Cognitive Dysfu...",117,"Chiba, Japan., Chiba, Japan.","ALS, ALS-bi, ALS-cbi, ALS-ci, Biomarker, Cogni...",1
35263201,Several studies have shown that the response o...,Predicting quality of life outcomes in childre...,Expert review of neurotherapeutics,22,2022-04,"Bhosle Rajesh, Aditya Grandhi, Das Sayan, Kris...","Adult, Behavior Therapy, Child, Humans, Migrai...",0,"Basel, Switzerland., Basel, Switzerland., Base...","adolescents, children, lifestyle modification,...",1
35589001,The development of perinatal brain connectivit...,Predicting age and clinical risk from the neon...,NeuroImage,257,2022-08-15,"Oda Hitoshi, Tsujinaka Ryo, Fukuda Shiho, Sawa...","Brain, Connectome, Diffusion Magnetic Resonanc...",0,"United States., United States., United States....",None,0
33116427,Poor adherence to appropriately prescribed med...,Self-Report for Measuring and Predicting Medic...,Patient preference and adherence,14,2020,"Li Ting-Ting, Xiong Liu-Lin, Huang Jin, Wen So...",None,122,"Aichi, Japan., Aichi, Japan., Aichi, Japan., A...","attitude to health, drug monitoring, health be...",0
35868240,We only remember a fraction of what we see-inc...,Predicting visual memory across images and wit...,Cognition,227,2022-Oct,"Ukaegbe Onyinyechi C, Holt Brooke E, Keator Ly...","Attention, Humans, Memory, Mental Recall, Neur...",0,"TX, USA., TX, USA., TX, USA., TX, USA., TX, US...","Memorability, Recognition memory, Sustained at...",0
34218343,"Within Europe, death rates due to COVID-19 var...",Longitudinal variability in mortality predicts...,European journal of epidemiology,36,2021-Jun,"Depannemaecker Damien, Destexhe Alain, Jirsa V...","Aged, COVID-19, Europe, Humans, Longitudinal S...",17,"3000, Belgium., 3000, Belgium., 3000, Belgium....","All-cause mortality, COVID-19, Corona, Influen...",0


In [141]:
n_neighbors = 10

KNN_a = NearestNeighbors(n_neighbors)
KNN_a.fit(tfidf_abstract)

NNs_a = KNN_a.kneighbors(input_tfidf_abstract,return_distance=True)

df.iloc[list(NNs_a[1][0]), :][['abstract']].value_counts()

abstract                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                